In [1]:
!apt-get update -qq
!apt-get install -y ffmpeg

!pip install -q \
  yt-dlp \
  pydub \
  numpy \
  scipy \
  soundfile \
  pandas \
  ipywidgets


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 121 not upgraded.


In [2]:
!pip install -q noisereduce

import noisereduce as nr


In [3]:
!rm -rf output
!rm -f *.py *.json *.txt


In [4]:
from google.colab import files
uploaded = files.upload()
print("Uploaded:", list(uploaded.keys()))


Saving config.json to config.json
Saving manual_transcription_colab.py to manual_transcription_colab.py
Saving pipeline.py to pipeline.py
Saving urls.txt to urls.txt
Uploaded: ['config.json', 'manual_transcription_colab.py', 'pipeline.py', 'urls.txt']


In [5]:
!python pipeline.py --urls urls.txt --config config.json --speaker speaker1


2026-02-09 17:31:21,071 - INFO - Processing 1 videos
2026-02-09 17:31:21,071 - INFO - 
2026-02-09 17:31:21,071 - INFO - Video 1/1
2026-02-09 17:31:21,071 - INFO - ============================================================

2026-02-09 17:31:21,071 - INFO - Processing video: https://youtu.be/qEnqmVXeDBc?si=SVKulCt4O8bxVzij
2026-02-09 17:31:21,071 - INFO - Downloading audio from: https://youtu.be/qEnqmVXeDBc?si=SVKulCt4O8bxVzij
[youtube] Extracting URL: https://youtu.be/qEnqmVXeDBc?si=SVKulCt4O8bxVzij
[youtube] qEnqmVXeDBc: Downloading webpage
[youtube] qEnqmVXeDBc: Downloading android vr player API JSON
[info] qEnqmVXeDBc: Downloading 1 format(s): 251
[download] Destination: output/raw_audio/qEnqmVXeDBc.webm
[download] 100% of    2.66MiB in 00:00:00 at 8.78MiB/s
[ExtractAudio] Destination: output/raw_audio/qEnqmVXeDBc.wav
Deleting original file output/raw_audio/qEnqmVXeDBc.webm (pass -k to keep)
2026-02-09 17:31:24,300 - INFO - Converting to required audio format...
2026-02-09 17:31:24

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
from IPython.display import Audio, display
import pandas as pd

df = pd.read_csv("output/metadata.csv", sep="|")

for _, row in df.head(5).iterrows():
    print(row["audio_filename"])
    display(Audio(f"output/wavs/{row['audio_filename']}"))


audio_000000.wav


audio_000001.wav


audio_000002.wav


audio_000003.wav


audio_000004.wav


In [10]:
from manual_transcription_colab import ManualTranscriptionTool, InteractiveTranscriber

# Initialize tool
tool = ManualTranscriptionTool()

# Start interactive session
transcriber = InteractiveTranscriber(tool)
transcriber.start()

HTML(value='\n        <div style="\n            background:#ecfdf5;\n            border-left:6px solid #22c55e…

TRANSCRIPTION PROGRESS
Total segments: 16
✅ Completed: 16 (100.0%)
⏳ Remaining: 0 (0.0%)


In [12]:
from manual_transcription_colab import ManualTranscriptionTool
tool = ManualTranscriptionTool()
tool.show_progress()


✅ Loaded 0 segments needing transcription
TRANSCRIPTION PROGRESS
Total segments: 16
✅ Completed: 16 (100.0%)
⏳ Remaining: 0 (0.0%)


In [13]:
tool.export_corrected_metadata("output/metadata_final.csv")


✅ Exported corrected metadata to: output/metadata_final.csv


In [14]:
from pathlib import Path
import zipfile
import pandas as pd

def export_dataset_zip(
    wavs_dir="output/wavs",
    metadata_csv="output/metadata_final.csv",
    zip_name="dataset_export.zip"
):
    wavs_dir = Path(wavs_dir)
    metadata_csv = Path(metadata_csv)

    assert wavs_dir.exists(), "❌ wavs directory not found"
    assert metadata_csv.exists(), "❌ metadata CSV not found"

    with zipfile.ZipFile(zip_name, "w", zipfile.ZIP_DEFLATED) as zipf:
        # Add metadata
        zipf.write(metadata_csv, arcname="metadata_final.csv")

        # Add wav files
        for wav in wavs_dir.glob("*.wav"):
            zipf.write(wav, arcname=f"wavs/{wav.name}")

    size_mb = Path(zip_name).stat().st_size / (1024 * 1024)
    print(f"✅ Exported: {zip_name}")
    print(f"📦 Size: {size_mb:.2f} MB")
    print(f"🎵 WAV files: {len(list(wavs_dir.glob('*.wav')))}")

# =========================
# USAGE
# =========================

# 1️⃣ Export corrected CSV (you already do this)
tool.export_corrected_metadata("output/metadata_final.csv")

# 2️⃣ Create ZIP
export_dataset_zip(
    wavs_dir="output/wavs",
    metadata_csv="output/metadata_final.csv",
    zip_name="dataset_export.zip"
)


✅ Exported corrected metadata to: output/metadata_final.csv
✅ Exported: dataset_export.zip
📦 Size: 2.98 MB
🎵 WAV files: 16


In [15]:
from google.colab import files
files.download("dataset_export.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
import zipfile

with zipfile.ZipFile("dataset_export.zip") as z:
    print(z.namelist()[:10])


['metadata_final.csv', 'wavs/audio_000010.wav', 'wavs/audio_000013.wav', 'wavs/audio_000014.wav', 'wavs/audio_000006.wav', 'wavs/audio_000008.wav', 'wavs/audio_000002.wav', 'wavs/audio_000007.wav', 'wavs/audio_000003.wav', 'wavs/audio_000009.wav']
